In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from tqdm import tqdm
import tensorflow as tf

In [2]:
a1=pd.read_csv('testing.csv')
a2=pd.read_csv('trainning.csv')

In [3]:
accept1=[]
accept2=[]

for key in a1.keys():
    if len(np.unique(a1[key]))>1:
        accept1.append(key)
for key in a2.keys():
    if len(np.unique(a2[key]))>1:
        accept2.append(key) 


In [4]:
def normarlize(length):
    return (length-length.mean())/length.std(),length.mean(),length.std()

In [6]:
b1=list()
b2=list()
des=dict()
accept2=np.delete(accept2,np.argwhere(accept2=='label'))
for key in tqdm(sorted(accept2)):
    if a2[key].dtype=='int64' or a2[key].dtype=='float64':
        nm,mean,std=normarlize(a2[key])
        nm=np.expand_dims(nm.tolist(),0).T
        des[key]=[mean,std]
    elif a2[key].dtype=='O':
        uni=a2[key].unique()
        l=len(uni)
        if '-1' in uni:
            l-=1
        nm=np.zeros((len(a2[key]),l))
        uni=np.delete(uni,np.argwhere(uni=='-1'))
        uni=dict(zip(uni,list(range(len(uni)))))
        des[key]=uni
        for i,n in enumerate(a2[key]):
            try:
                nm[i,uni[n]]=1
            except:
                continue  
    else:
        continue
    b2.append(nm)    
    
for key in tqdm(sorted(accept2)):
    if a1[key].dtype=='int64' or a1[key].dtype=='float64':
        nm=(a1[key]-des[key][0])/des[key][1]
        nm=np.expand_dims(nm.tolist(),0).T
    elif a1[key].dtype=='O':
        nm=np.zeros((len(a1[key]),len(des[key].keys())))
        for i,n in enumerate(a1[key]):
            try:
                nm[i,des[key][n]]=1
            except:
                continue  
    else:
        continue
    b1.append(nm)  

100%|██████████| 45/45 [00:00<00:00, 296.57it/s]


In [7]:
data_train=np.concatenate(b2,axis=1)
data_test=np.concatenate(b1,axis=1)

In [12]:
svclassifier =SVC(kernel='linear',verbose=True,C=5)
svclassifier.fit(data_train,a2.label)

[LibSVM]

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=True)

In [13]:
pred=svclassifier.predict(data_train)
print(classification_report(a2.label, pred))
pred=svclassifier.predict(data_test)
print(classification_report(a1.label, pred))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91     50400
           1       1.00      0.43      0.60     17545

   micro avg       0.85      0.85      0.85     67945
   macro avg       0.92      0.72      0.76     67945
weighted avg       0.88      0.85      0.83     67945

              precision    recall  f1-score   support

           0       0.83      1.00      0.91     21600
           1       1.00      0.43      0.60      7520

   micro avg       0.85      0.85      0.85     29120
   macro avg       0.92      0.71      0.75     29120
weighted avg       0.88      0.85      0.83     29120



In [8]:
batch_size = 1000
layer_ids = ['hidden1','hidden2','out']
layer_sizes = [data_train.shape[1], 200,150, 2]
train_inputs = tf.placeholder(tf.float32, shape=[None, layer_sizes[0]], name='train_inputs')
train_labels = tf.placeholder(tf.int32, shape=[None], name='train_labels')
for idx, lid in enumerate(layer_ids):
    with tf.variable_scope(lid):
        w = tf.get_variable('weights',shape=[layer_sizes[idx], layer_sizes[idx+1]],
                            initializer=tf.truncated_normal_initializer(stddev=0.05))
        b = tf.get_variable('bias',shape= [layer_sizes[idx+1]],
                            initializer=tf.random_uniform_initializer(-0.1,0.1))
h = train_inputs
for lid in layer_ids:
    with tf.variable_scope(lid,reuse=True):
        w, b = tf.get_variable('weights'), tf.get_variable('bias')
        if lid != 'out':
            h = tf.nn.relu(tf.matmul(h,w)+b,name=lid+'_output')
        else:
            h = tf.nn.xw_plus_b(h,w,b,name=lid+'_output')
tf_predictions = tf.nn.softmax(h, name='predictions')
tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(train_labels,depth=2), logits=h),name='loss')
tf_learning_rate = tf.placeholder(tf.float32, shape=None, name='learning_rate')
optimizer = tf.train.AdamOptimizer(learning_rate=0.001,).minimize(tf_loss)
sess=tf.Session()
sess.run(tf.global_variables_initializer())

Load model fail


In [11]:
step=0
saver=tf.train.Saver(max_to_keep=1)
try:
    checkpoint = tf.train.latest_checkpoint('checkpoint')
    saver.restore(sess, checkpoint)
    step=int(checkpoint.split('-')[-1])+1
    print('Load complete at step: ',str(step))
except Exception:
    print('Load model fail')
    step=0
for epoch in range(step,2000):
    L=0
    for iters in range(data_train.shape[0]//batch_size):
        _,l=sess.run([optimizer,tf_loss],feed_dict={train_inputs:data_train[iters*batch_size:(iters+1)*batch_size],train_labels:a2.label[iters*batch_size:(iters+1)*batch_size]})
        L+=l
    print('Step: {:d} loss: {:f} '.format(epoch,L))
    if (epoch+1)%10==0:
        saver.save(sess,'checkpoint/model',global_step=epoch)
        count=0
        for iters in range(data_test.shape[0]//batch_size):
            predict=sess.run(tf_predictions,feed_dict={train_inputs:data_test[iters*batch_size:(iters+1)*batch_size]})
            predict=np.argmax(predict,axis=1)
            count+=sum(predict==a1.label[iters*batch_size:(iters+1)*batch_size])
        print('Accuracy: ',str(count/len(data_test)))

INFO:tensorflow:Restoring parameters from checkpoint/model-999
Load complete at step:  1000
Step: 1000 loss: 20.189390 
Step: 1001 loss: 40.842369 
Step: 1002 loss: 18.268194 
Step: 1003 loss: 19.314547 
Step: 1004 loss: 19.758511 
Step: 1005 loss: 19.099962 
Step: 1006 loss: 18.420989 
Step: 1007 loss: 17.300523 
Step: 1008 loss: 17.538438 
Step: 1009 loss: 17.052180 
Accuracy:  0.8737980769230769
Step: 1010 loss: 16.837553 
Step: 1011 loss: 16.566855 
Step: 1012 loss: 16.498178 
Step: 1013 loss: 16.452642 
Step: 1014 loss: 16.441703 
Step: 1015 loss: 16.412061 
Step: 1016 loss: 16.371610 
Step: 1017 loss: 16.385583 
Step: 1018 loss: 16.326042 
Step: 1019 loss: 16.383528 
Accuracy:  0.8769917582417582
Step: 1020 loss: 16.309437 
Step: 1021 loss: 16.487015 
Step: 1022 loss: 16.319016 
Step: 1023 loss: 16.422553 
Step: 1024 loss: 16.233277 
Step: 1025 loss: 16.333511 
Step: 1026 loss: 16.163591 
Step: 1027 loss: 16.262859 
Step: 1028 loss: 16.206428 
Step: 1029 loss: 16.351854 
Accuracy

Step: 1262 loss: 17.406027 
Step: 1263 loss: 17.173646 
Step: 1264 loss: 17.200244 
Step: 1265 loss: 17.370348 
Step: 1266 loss: 17.144940 
Step: 1267 loss: 17.164632 
Step: 1268 loss: 17.438784 
Step: 1269 loss: 17.055088 
Accuracy:  0.8755494505494505
Step: 1270 loss: 17.153149 
Step: 1271 loss: 16.568653 
Step: 1272 loss: 17.390942 
Step: 1273 loss: 17.897695 
Step: 1274 loss: 17.658264 
Step: 1275 loss: 16.982225 
Step: 1276 loss: 17.165690 
Step: 1277 loss: 17.610783 
Step: 1278 loss: 16.341433 
Step: 1279 loss: 16.363711 
Accuracy:  0.8750343406593407
Step: 1280 loss: 16.351997 
Step: 1281 loss: 16.487973 
Step: 1282 loss: 16.265102 
Step: 1283 loss: 15.944338 
Step: 1284 loss: 16.014268 
Step: 1285 loss: 16.186053 
Step: 1286 loss: 15.916451 
Step: 1287 loss: 15.852213 
Step: 1288 loss: 15.859014 
Step: 1289 loss: 15.815665 
Accuracy:  0.8789148351648352
Step: 1290 loss: 15.803894 
Step: 1291 loss: 15.785903 
Step: 1292 loss: 15.746865 
Step: 1293 loss: 15.714826 
Step: 1294 los

Step: 1528 loss: 17.560623 
Step: 1529 loss: 17.135113 
Accuracy:  0.8744505494505495
Step: 1530 loss: 17.096714 
Step: 1531 loss: 17.054730 
Step: 1532 loss: 16.984282 
Step: 1533 loss: 16.955055 
Step: 1534 loss: 16.935493 
Step: 1535 loss: 16.913369 
Step: 1536 loss: 16.886785 
Step: 1537 loss: 16.822911 
Step: 1538 loss: 16.821576 
Step: 1539 loss: 16.641502 
Accuracy:  0.8747596153846153
Step: 1540 loss: 17.399381 
Step: 1541 loss: 16.725178 
Step: 1542 loss: 16.777952 
Step: 1543 loss: 16.553013 
Step: 1544 loss: 16.530762 
Step: 1545 loss: 16.464732 
Step: 1546 loss: 16.452113 
Step: 1547 loss: 16.437690 
Step: 1548 loss: 16.488995 
Step: 1549 loss: 16.385324 
Accuracy:  0.8767857142857143
Step: 1550 loss: 16.374429 
Step: 1551 loss: 16.341505 
Step: 1552 loss: 16.315872 
Step: 1553 loss: 16.308138 
Step: 1554 loss: 16.295424 
Step: 1555 loss: 16.281165 
Step: 1556 loss: 16.241115 
Step: 1557 loss: 16.256019 
Step: 1558 loss: 16.235817 
Step: 1559 loss: 16.228495 
Accuracy:  0.8

Step: 1793 loss: 15.584761 
Step: 1794 loss: 15.641959 
Step: 1795 loss: 15.454220 
Step: 1796 loss: 15.461348 
Step: 1797 loss: 15.386760 
Step: 1798 loss: 15.501525 
Step: 1799 loss: 15.802795 
Accuracy:  0.8756868131868132
Step: 1800 loss: 15.851270 
Step: 1801 loss: 15.962657 
Step: 1802 loss: 18.335429 
Step: 1803 loss: 17.272947 
Step: 1804 loss: 16.689483 
Step: 1805 loss: 16.589290 
Step: 1806 loss: 16.375030 
Step: 1807 loss: 16.462237 
Step: 1808 loss: 17.925519 
Step: 1809 loss: 16.896526 
Accuracy:  0.8741414835164835
Step: 1810 loss: 16.570686 
Step: 1811 loss: 16.400326 
Step: 1812 loss: 16.275594 
Step: 1813 loss: 16.223461 
Step: 1814 loss: 16.258367 
Step: 1815 loss: 16.187939 
Step: 1816 loss: 16.272567 
Step: 1817 loss: 16.542487 
Step: 1818 loss: 16.560170 
Step: 1819 loss: 16.336544 
Accuracy:  0.8784684065934066
Step: 1820 loss: 16.226267 
Step: 1821 loss: 16.220530 
Step: 1822 loss: 16.141827 
Step: 1823 loss: 16.267190 
Step: 1824 loss: 16.212421 
Step: 1825 los

In [15]:
count=0
for iters in range(data_test.shape[0]//batch_size):
    predict=sess.run(tf_predictions,feed_dict={train_inputs:np.reshape(data_test[iters*batch_size:(iters+1)*batch_size],[batch_size,-1])})
    predict=np.argmax(predict,axis=1)
    count+=sum(predict==a1.label[iters*batch_size:(iters+1)*batch_size])

In [18]:
predict=sess.run(tf_predictions,feed_dict={train_inputs:data_train})
predict=np.argmax(predict,axis=1)
print(classification_report(a2.label, predict))
predict=sess.run(tf_predictions,feed_dict={train_inputs:data_test})
predict=np.argmax(predict,axis=1)
print(classification_report(a1.label, predict))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93     50400
           1       1.00      0.57      0.72     17545

   micro avg       0.89      0.89      0.89     67945
   macro avg       0.93      0.78      0.83     67945
weighted avg       0.90      0.89      0.88     67945

              precision    recall  f1-score   support

           0       0.87      1.00      0.93     21600
           1       0.99      0.56      0.71      7520

   micro avg       0.88      0.88      0.88     29120
   macro avg       0.93      0.78      0.82     29120
weighted avg       0.90      0.88      0.87     29120

